<a href="https://colab.research.google.com/github/rumeysayuk/sentimentAnalysis/blob/main/sentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import numpy as np
import pandas as pd

In [137]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [138]:
dataset = pd.read_csv("hepsiburada.csv")

In [139]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [140]:
target=dataset["Rating"].values.tolist()
data=dataset["Review"].values.tolist()

In [141]:
#model eğitileceği zaman 2 ye bölünür.eğitim ve test diye.Eğitim seti üzerinde eğitip,eğitim bitince test seti üzerinde test ederiz.
#%80 eğitim %20 test

In [142]:
cutoff=int(len(data)*0.80)
x_train,x_test=data[:cutoff],data[cutoff:]
y_train,y_test=target[:cutoff],data[cutoff:]

In [143]:
x_train[500] #500.yorum

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [144]:
y_train[500] #olumlu ise 1 degilse 0

1

In [145]:
#Tokenlestirme asaması.Her yorum kelimelere ayrılacak ve her yorrum kelimelere ayrılcak ve her kelimeye karşilik farklı sayılar denk gelcek

In [146]:
num_words=10000 #kelime hazne sayımız.en sık gecen 10.000 kelimeyi aldık.keras ile yapıcaz tokenlestirmeyi.
tokenizer=Tokenizer(num_words=num_words)

In [147]:
tokenizer.fit_on_texts(data) #tokenlestirme islemi

In [ ]:
tokenizer.word_index

In [149]:
x_train_tokens=tokenizer.texts_to_sequences(x_train) # egitim seti tokenlestirildi

In [150]:
x_train[500]

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [151]:
print(x_train_tokens[500])

[4, 57, 97, 26, 40, 112, 29, 47, 59, 130, 100, 20, 51, 63, 4, 11, 838, 18, 5, 587, 273, 157, 5, 369, 1396, 17, 303, 608, 381]


In [152]:
x_test_tokens=tokenizer.texts_to_sequences(x_test) #test data seti tokenlestirildi.Tokenlestirmeler tamamlandı

In [153]:
#tum yorumları belirttigimiz boyuta getirecegiz eksik kalırsa 0 ekleyecegiz.

In [154]:
num_tokens=[len(tokens)for tokens in x_train_tokens+x_test_tokens] 
num_tokens=np.array(num_tokens)

In [155]:
#bir yorumda kac tokens oldugunu bulacagız her yorumun üzerinden gecip token sayısı alındı

In [156]:
np.mean(num_tokens) #ort. olarak bir yorumda kaç token olduğunu hesaplıyor

20.744703220162876

In [157]:
np.max(num_tokens) #en çok tokeni olan yorum

295

In [158]:
np.argmax(num_tokens) #en uzun yorumun indexi bunu da x[train] icinde ararsak yoruma ulasırız
x_train[21941]

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [159]:
#tum yorumlarun boyunu esitlemek icin padding uygulanır(yorum dengesizligini ortadan kaldırırız boylece)
max_tokens=np.mean(num_tokens)+2*np.std(num_tokens)
max_tokens=int(max_tokens)
max_tokens #networke 59 tokenli yorumlar verilecek

59

In [105]:
x_train_pad=pad_sequences(x_train_tokens,maxlen=max_tokens) #her yorum esit hale geldi padding eklenerek

In [160]:
x_test_pad=pad_sequences(x_test_tokens,maxlen=max_tokens)

In [161]:
x_train_pad.shape #tum yorumlar ve boyutlar

(194797, 59)

In [162]:
x_test_pad.shape

(48700, 59)

In [163]:
np.array(x_train_tokens[500]) #59 degil uzunlugu burada

array([   4,   57,   97,   26,   40,  112,   29,   47,   59,  130,  100,
         20,   51,   63,    4,   11,  838,   18,    5,  587,  273,  157,
          5,  369, 1396,   17,  303,  608,  381])

In [164]:
x_train_pad[500] #59' a tamamlandı

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    4,   57,   97,
         26,   40,  112,   29,   47,   59,  130,  100,   20,   51,   63,
          4,   11,  838,   18,    5,  587,  273,  157,    5,  369, 1396,
         17,  303,  608,  381], dtype=int32)

In [165]:
idx=tokenizer.word_index
inverse_map=dict(zip(idx.values(),idx.keys()))

In [166]:
#yorumu verince yorumun text halini donduren fonksiyon
def tokens_to_string(tokens):
  words=[inverse_map[token] for token in tokens if token!=0]
  text=' '.join(words)
  return text

In [167]:
x_train[500]

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [168]:
tokens_to_string(x_train_tokens[500])

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım bu yüzden tereddütsüz aldım alacak olanlara öneririm'

In [177]:
#GRU'nun ürettiği sayı 1 e ykaınsa olumlu ,sıfıra yakınsa olumsuz 
model = Sequential()
embedding_size = 50
model.add(Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_tokens,
                    name='embedding_layer'))  # 10000 50 boynutunda bir matris oluşturmuş olduk
 #her kelimeye denk gelen 50 uzunlugunda vektorler olusturulcak.Vektorleri w2vec yada glove ile degilde rastgele olusturarak egitmeyi deniyoruz

In [172]:
#GRu ve CudnnGru ile denenebiliyor.Her layer digerinin outputunu kullanıyor

In [178]:
model.add(CuDNNGRU(units=16, return_sequences=True))
model.add(CuDNNGRU(units=8, return_sequences=True))  # 8 tane noronlu sinir agi
model.add(CuDNNGRU(units=4))  # 8 tane noronlu sinir agi
model.add(Dense(1, activation="sigmoid"))

In [183]:
optimizer=Adam(learning_rate=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [185]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
cu_dnngru_25 (CuDNNGRU)      (None, 59, 16)            3264      
_________________________________________________________________
cu_dnngru_26 (CuDNNGRU)      (None, 59, 8)             624       
_________________________________________________________________
cu_dnngru_27 (CuDNNGRU)      (None, 4)                 168       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [188]:
model.fit(np.array(x_train_pad), np.array(y_train), epochs=5, batch_size=256) #egitim yapılıyor

RuntimeError: ignored

In [ ]:
result=model.evaluate(x_test_pad,y_test)

In [ ]:
result[1]